# Programación 2 - TUIA
---
## Trabajo Práctico - Tabla Hash




- Comisión:
- Apellido y Nombre (integrante 1): Antuña, Franco
- Apellido y Nombre (integrante 2): Añaños, Diego

### Implementación de Tabla Hash para control de stock

**Objetivo:** En este trabajo, debe implementar un programa que ayude a llevar el control de stock de un supermercado.

**Descripción del problema:** Implemente el programa para el control de stock de un supermercado usando una Tabla Hash. La misma se utilizará para almacenar información sobre los productos disponibles en el supermercado y la cantidad de unidades disponibles. Se debe permitir la inserción, eliminación y búsqueda de productos. La Tabla Hash debe resolver sus colisiones con encadenamiento. Además se deben implementar algunas funciones de estadísticas de uso de la tabla.

Tenga en cuenta que el supermercado vende normalmente más de $40000$ productos. Cada producto tiene un código alfanumérico de $10$ caracteres, por ejemplo "AAN1235465", "123ABCDE12".

### Requerimientos
- Escriba una función apropiada para transformar las claves del problema a valores numéricos.
- Implemente una tabla hash para almacenar los productos disponibles. Debera:

  - incluir el método constructor.
  - utilizar una función hash apropiada - puede elegir el método del resto, el método de la multiplicación o bien proponer otro que haya investigado.
  - debe permitir la inserción, eliminación y búsqueda de productos en la tabla hash.
  - la tabla hash debe soportar resolución de colisiones mediante encadamiento, puede utilizar para ello una lista enlazada o una lista de Python, a su preferencia.
  
- Además debe implementar los siguientes métodos:

  - `__len__` Este método sobrecarga la función incluida `len`. Debe devolver la cantidad de elementos insertados en la tabla.
  - `_load_factor`: Este método debe calcular y devolver el factor de carga de la tabla hash.
  - `_longest_list`: Este método debe calcular y devolver el tamaño de la lista más larga.

- **Asegúrese de respetar la interfaz de los tests**, el TP será probado con ese formato.

### Entregables
El ejercicio debe ser resuelto de manera individual y deberá entregarse por medio del CampusV de la materia. Se debe entregar este Colab completo y funcional, con las siguientes preguntas contestadas en una celda de texto:
1. ¿Que eligió como función de hash? Explique porque considera que tomo una buena decisión.
2. ¿Que valor eligió como tamaño? Explique porque considera que tomo una buena decisión.
3. ¿Puede ser un problema utilizar un tamaño de tabla fijo? Justifique.
4. ¿Cual es la diferencia entre un diccionario y una tabla hash?
5. ¿Utilizo algún invariante de objeto? ¿Por qué?

### Criterios de evaluación
- Funcionamiento adecuado del programa y cumplimiento de los requisitos establecidos basado en el cumplimiento de tests de caja negra.
- Claridad y organización del código. Recuerde utilizar comentarios.
- Calidad del informe entregado.

1) Elegimos la funcion del resto por la simplicidad de la misma, en las pruebas utilizamos la de multiplicacion, y otras como Cyclic redundancy check y BUZ, pero encontramos problemas en la busqueda o en el insert.

2) Consideramos un tamaño de 64783, probamos multiples valores entre 60000 y 70000 mientras optimizabamos el factor de carga entre 0,6 y 0,75 y el tamaño de la mayor lista. El valor seleccionado nos dio resultados que consideramos bastante buenos.

3) Depende, si la cantidad de valores es conocida, entonces no. Pero en caso de desconocer la cantidad de valores, o que en el futuro se aumenten sin aumentar el tamaño de la lista, si seria un problema. En las pruebas tambien hicimos tests con cantidades de valores mayor a las del tamaño de la lista, eso genero que las listas de mayor longitud sean demasiado extensas, el factor era muy elevado y aumentaba el tiempo de ejecucion a medida que aumentaba la cantidad de valores.

4) Si bien ambos almacenan claves con valores, no se aplican de la misma manera, la tabla hash utiliza una funcion para almacenar en el mismo casillero multiples claves, esto acelera el acceso a los datos. Los diccionarios son estrucutras mas generales.

5) No, y no consideramos necesario ya que establecimos multiples pruebas para verificar que la logica del codigo funcione como se espera.

In [8]:
from typing import Any

class Clave():
    """
    Almacena la clave real a ser hasheada,
    usando la representación que se desee,
    y encapsula el algoritmo de hash elegido.
    """

    def __init__(self, key: str):
        self.key = key

    def to_int(self) -> int:
        """
        Solo recibe parametro self, utiliza el llamado a self.key y devuelve el valor de la string en int
        Funcion resuelta a partir de la pagina 4 del apunte de la materia
        """
        nat_valor = 0
        base = 10 # se toma una base decimal 10 para utilizar numeros del 0 al 9
        largo = len(self.key)
        for i in range(largo):
            key_int = ord(self.key[i])  # Obtén el valor numérico del carácter (por ejemplo, valor ASCII)
            nat_valor += key_int * (base ** (largo - 1 - i))
        return nat_valor

    def __eq__(self, other) -> bool:
        """
        Decide si dos claves son iguales
        """
        return self.key == other.key

In [9]:
# No es necesario cambiar nada en esta celda! Esta celda solo define la interfaz
# del TAD Diccionario.
from typing import Any

class Diccionario():
  """
  Interfaz del TAD Diccionario
  - insert(key, value) - Inserta un elemento con clave key y valor value en el diccionario.
      Si la clave ya se encuentra en el diccionario, debe reemplazar el value anterior por el nuevo.
  - search(key) - Devuelve la el value asociado con la clave key, o muestra un mensaje de error si la clave no se encuentra.
  - delete(key) - Elimina el elemento con clave key del diccionario.
  """

  def insert(self, key: Clave, data: Any) -> None:
    pass

  def search(self, key: Clave) -> Any:
    pass

  def delete(self, key: Clave) -> None:
    pass


In [10]:
class HashTable(Diccionario):
    """
    Implementación de Diccionario con tabla hash
    de tamaño fijo y resolución de colisiones
    por encadenamiento
    """

    def __init__(self, size: int):
        """
        El constructor inicializa una tabla de tamaño `size` que recibe
        como parametro. Utiliza el tipo lista incluido en Python
        """
        ### COMPLETAR AQUI CON SU CODIGO.
        self.size = size
        self.table = [None] * size

    def funcion_hash(self, key: Clave):
        """
        Recibe como parametro la clave convertida a int y devuevle el
        modulo entre el valor recibido y el tamaño de la tabla
        """
        return key % self.size

    def insert(self, key: Clave, data: Any) -> None:
        """
          Implementacion del metodo insert, recibe como parametros el objeto Clave
        y dato al que hace referencia.

          Este metodo asigna las claves al bucket correspondiente
        en función de su valor hash y gestiona tanto la inserción de nuevas
        claves como la actualización de claves existentes en la tabla hash.

          No retorna un resultado.

          Bucket: contenedor o compartimento utilizado para almacenar un conjunto de elementos.
        """
        ### COMPLETAR AQUI CON SU CODIGO
        key_hash = self.funcion_hash(key.to_int()) # llamo al modulo to_int(), y le paso el resultado a la funcion hash
        if self.table[key_hash] is None:
            self.table[key_hash] = [(key, data)]
            return
        elif self.table[key_hash] is not None: # verifico la existencia de la key dentro del valor hash, de existir se actualiza el dato
            for rango in range(0, len(self.table[key_hash])):
                if self.table[key_hash][rango][0] == key:
                    self.table[key_hash][rango] = [(key, data)]
                    return
        self.table[key_hash].append((key, data))

    def search(self, key: Clave) -> Any | None:
        """
         Implementacion del metodo search. Recibe como parametro la clave y retorna
        el valor asociado en caso de existir, o None en caso contrario.

          Calcula el hash de la clave que se está buscando y verifica si la
        posición calculada por el hash está vacía. De ser asi se devuelve None para
        indicar que la clave no se encuentra en la tabla.
          Si hay entradas en ese bucket, se recorren para buscar una clave específica.
        Cuando se encuentra la coincidencia, se devuelve el valor asociado a esa clave.


        """
        ### COMPLETAR AQUI CON SU CODIGO.
        key_hash = self.funcion_hash(key.to_int())
        if self.table[key_hash] == None:
            return None
        else:
            for k, valor in self.table[key_hash]:
              if k == key:
                return valor

    def delete(self, key: Clave) -> None:
        """
          Implementacion del metodo delete. Recibe como parametro la key de la clase Clave y no devuelve valor.

          Se verifica si hay una entrada en esa posición de la tabla. Si hay entradas (tuplas),
        se recorren para encontrar la tupla que tiene la misma clave que se desea eliminar.

          Cuando se encuentra la coincidencia, esa tupla se elimina del bucket.
        """
        ### COMPLETAR AQUI CON SU CODIGO.
        key_hash = self.funcion_hash(key.to_int())
        if self.table[key_hash] is not None:
            for i, (k, _) in enumerate(self.table[key_hash]): #  con el _ indico que no estoy interesados en la segunda parte de la tupla
                if k == key:
                    del self.table[key_hash][i]

    def __len__(self) -> int:
        """
        Este método sobrecarga la función `len`.

        Debe devolver la cantidad de elementos insertados en la tabla.

        Si la tabla esta vacía, devuelve cero, sino inicializa un contador en cero.
        Luego, recorre cada bucket en la tabla hash y suma la cantidad de elementos al contador.
        Finalmente, devuelve el valor del contador.
        """
        ### COMPLETAR AQUI CON SU CODIGO.
        if self.table is None:
          return 0
        contador = 0
        for lista in self.table:
            if lista is not None:
                contador += len(lista)
        return contador

    def _load_factor(self) -> float:
        """
        Este método debe devolver el factor de carga de la tabla.

        Obtiene la cantidad de elementos almacenados en la tabla y la capacidad total
        de la tabla y calcula el factor de carga dividiendo la cantidad de elementos
        almacenados por la capacidad total.
        """
        ### COMPLETAR AQUI CON SU CODIGO.
        almacenado = 0  # Inicializa la cantidad de elementos almacenados
        almacenado = len(self)
        capacidad = self.size  # Obtiene la capacidad total de la tabla
        load_factor = almacenado / capacidad  # Calcula el factor de carga
        return load_factor


    def _longest_list(self) -> int:
        """
        Este método debe devolver la longitud de la lista mas larga de la tabla
        """
        ### COMPLETAR AQUI CON SU CODIGO.

        longitud_max = 0
        for bucket in self.table:
            if bucket is not None:
                longitud_max = max(longitud_max, len(bucket))
        return longitud_max


    def __str__(self) -> str:
        """
        Mostrar la cantidad de elementos de la tabla (__len__), el factor de carga (_load_factor)
        y la longitud de la lista mas larga (_longest_list)
        """
        ### COMPLETAR AQUI CON SU CODIGO.
        elements_count = len(self)
        load_factor = self._load_factor()
        longest_list_length = self._longest_list()

        return f"Elementos en la tabla: {elements_count}\n" \
            f"Factor de carga: {load_factor}\n" \
            f"Longitud de la lista más larga: {longest_list_length}"
    

## Código de Prueba

La siguiente celda deberia funcionar como un test básico de funcionamiento. Si su implementación tiene sentido, debería poder correr la siguiente celda sin problemas. En caso de que la prueba funcione, ejecutar el siguiente código no imprime nada. En caso de problemas, mostrara un error.

**Nota**: El funcionamiento de este código no quiere decir que la solución sea correcta. La prueba es muy básica ya que solo prueba insertar y eliminar un elemento.

**Nota: La instrucción `assert`**: El uso de assert en Python nos permite realizar comprobaciones. Si la expresión contenida dentro del mismo es False, se lanzará un error, mas concretamente un `AssertionError`. Por ejemplo, `assert 1==2` nos daría un `AssertionError`. Puede leer mas sobre esta instrucción en [el libro de Python](https://ellibrodepython.com/assert-python).



**ATENCION: NO MODIFIQUE ESTA CELDA** Si desea hacer más pruebas, hagalo en una celda nueva.

In [11]:
from random import choice, randrange

def generar_clave_random() -> Clave:
  """
  Genera una clave alfanumerica de 10 digitos al azar
  """
  chars = "1234567890QWERTYUIOPASDFGHJKLZXCVBNM"
  string = "".join(choice(chars) for _ in range(10))
  return Clave(string)

def generar_valor_random() -> int:
  """
  Devuelve un valor al azar entre 0 y 100 millones
  """
  return randrange(0, 100_000_000)

def prueba_de_funcionamiento(diccionario: Diccionario):
  # assert rompe el programa si la condición es falsa
  # esto es útil para verificar cosas que deberían ser ciertas

  clave = generar_clave_random()
  valor = generar_valor_random()

  diccionario.insert(clave, valor)

  assert len(diccionario) == 1

  assert diccionario.search(clave) == valor

  diccionario.delete(clave)

  assert len(diccionario) == 0

# Para probar esto, primero implemente to_int en Celda y la clase HashTable
prueba_de_funcionamiento(HashTable(100))

In [12]:
def prueba_de_funcionamiento(diccionario: Diccionario):
    # assert rompe el programa si la condición es falsa
    # esto es útil para verificar cosas que deberían ser ciertas
    for i in range(40000):
        clave = generar_clave_random()
        valor = generar_valor_random()
        diccionario.insert(clave, valor)

    assert len(diccionario) == 40000
    print("Test 1: pass")

    assert diccionario.search(clave) == valor
    print("Test 2: pass")

    diccionario.delete(clave)

    assert len(diccionario) == 39999
    print("Test 3: pass")

# Para probar esto, primero implemente to_int en Celda y la clase HashTable
prueba_de_funcionamiento(HashTable(65003))

Test 1: pass
Test 2: pass
Test 3: pass


In [14]:
import random

"""
Esto es un test donde evaluamos el rendimiento y los metodos de la tabla hash,
durante el proceso utilizamos multiples valores y codigos.
"""

keys = [''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ123456789', k=10)) for _ in range(40000)]
hash_table = HashTable(64783)
print(len(hash_table))
for key in keys:
    hash_table.insert(Clave(key), f"Valor asociado a {key}")
for _ in range(10):
    random_key = random.choice(keys)
    result = hash_table.search(Clave(random_key))
    if result is not None:
        print(f"Clave encontrada: {random_key}, Valor: {result}")
    else:
        print(f"Clave no encontrada: {random_key}")
result = hash_table.search(Clave("GGGGG"))
random_key = "GGGGG"
if result is not None:
    print(f"Clave encontrada: {random_key}, Valor: {result}")
else:
    print(f"Clave no encontrada: {random_key}")
for _ in range(10):
    random_key = random.choice(keys)
    hash_table.delete(Clave(random_key))
    print(f"Clave eliminada: {random_key}")

print(hash_table)

0
Clave encontrada: HQ2ULT5BU2, Valor: Valor asociado a HQ2ULT5BU2
Clave encontrada: NLKGENQ15K, Valor: Valor asociado a NLKGENQ15K
Clave encontrada: XT1TAS8T3D, Valor: Valor asociado a XT1TAS8T3D
Clave encontrada: GCPVI8WM53, Valor: Valor asociado a GCPVI8WM53
Clave encontrada: ZJSGTVNA8P, Valor: Valor asociado a ZJSGTVNA8P
Clave encontrada: S46D7ZIFE2, Valor: Valor asociado a S46D7ZIFE2
Clave encontrada: O5UZ6PV6WJ, Valor: Valor asociado a O5UZ6PV6WJ
Clave encontrada: UW61N1AMI1, Valor: Valor asociado a UW61N1AMI1
Clave encontrada: 6PR6UMNIWY, Valor: Valor asociado a 6PR6UMNIWY
Clave encontrada: NZOWUMQH19, Valor: Valor asociado a NZOWUMQH19
Clave no encontrada: GGGGG
Clave eliminada: XD8BBN74QT
Clave eliminada: 83PSQRX9L4
Clave eliminada: XQ1CE972GA
Clave eliminada: X41ZIH37FF
Clave eliminada: QKZDK4SILM
Clave eliminada: P3LDT3HBJG
Clave eliminada: 4JB21GKPAK
Clave eliminada: 8LK5FXS9ZS
Clave eliminada: DUQOGGZBXF
Clave eliminada: WESXDQSX3M
Elementos en la tabla: 39990
Factor de ca